In [7]:
import numpy as np
import math
import csv
import time

import queue
from statistics import mean
import matplotlib.pyplot as plt
import random
import statistics
import scipy.stats as sct
from tabulate import tabulate
from scipy.sparse import lil_matrix

In [ ]:
## OLD

N = 200
T = 100
arr_rate = 10
ps = 0.9
pb = 0.1
lambda_s = (arr_rate*ps)
lambda_b = (arr_rate*pb)
mu_s = 10
mu_b = 1

start_sim = time.time()

d1 = int(N + ((N*N)/(2*T)) - (N/2) + (N/T) + 1)
d2 = int((T)*(N/T) + 1)
#print(d1)
#print(d2)

q = np.zeros((d1,d1))
#print(q)

def access_q(n1,s1,b1,n2,s2,b2):
    global q
    q_dim = ((q.shape[0]-d1)/d2)
    if n1 > q_dim:
        #extra = int((n1 - (int(q.shape[0]/d1)-1))*d1)
        extra = d2
        
        cc = q.shape[1]
        q = np.append(q,np.zeros((extra,cc)), axis=0)
        cr = q.shape[0]
        q = np.append(q,np.zeros((cr,extra)), axis=1)
    elif n2 > q_dim:
        #extra = int((n2 - (int(q.shape[0]/d1)-1))*d1)
        extra = d2
        #print(extra)
        
        cc = q.shape[1]
        q = np.append(q,np.zeros((extra,cc)), axis=0)
        cr = q.shape[0]
        q = np.append(q,np.zeros((cr,extra)), axis=1)
    
    
    row = access_map[(s1,b1)]
    col = access_map[(s2,b2)]
    if (n1>0):
        row = d1 + ((n1-1)*d2) + access_map_v2[(s1,b1)]
    if (n2>0):
        col = d1 + ((n2-1)*d2) + access_map_v2[(s2,b2)]
    return((row,col))
    

access_map = dict()
i = 0
for s in range(N+1):
    for b in range(int(N/T)+1):
        if (s+(b*T)) <= N:
            access_map[(s,b)] = i
            i = i+1
            
access_map_v2 = dict()
i = 0
for s in range(N+1):
    for b in range(int(N/T)+1):
        if (s+(b*T)) > (N-T) and (s+(b*T)) <= N:
            access_map_v2[(s,b)] = i
            i = i+1

def init_transitions_arr(dim):
    global q
    for n in range(dim):
        for s in range(N+1):
            for b in range (N+1):
                if n > 0 and (s,b) in access_map_v2:
                    r,c = access_q(n,s,b, n+1,s,b)
                    q[r,c] = arr_rate
                elif n == 0:
                    if (s+(b*T)) == N:
                        r,c = access_q(0,s,b, 1,s,b)
                        q[r,c] = arr_rate
                    elif (s+(b*T)) <= (N-T):
                        r,c = access_q(0,s,b, 0,s+1,b)
                        q[r,c] = lambda_s
                        r,c = access_q(0,s,b, 0,s,b+1)
                        q[r,c] = lambda_b
                    elif (s+(b*T)) > (N-T) and (s+(b*T)) < N:
                        r,c = access_q(0,s,b, 0,s+1,b)
                        q[r,c] = lambda_s
                        r,c = access_q(0,s,b, 1,s,b)
                        q[r,c] = lambda_b
                        
def init_transitions_srv(dim):
    global q
    for n in range(dim):
        for s in range(N+1):
            for b in range (N+1):
                F = N - (b*T) - s
                
                if n > 0 and (s+(b*T)) == N:
                    K = min(n,T)
                    if s > 0:
                        r,c = access_q(n,s,b, n-1,s,b)
                        q[r,c] = q[r,c] + (s*mu_s*ps)

                        r,c = access_q(n,s,b, n,s-1,b)
                        q[r,c] = q[r,c] + (s*mu_s*pb)
                    if b > 0:
                        r,c = access_q(n,s,b, n-1,s,b)
                        q[r,c] = q[r,c] + (b*mu_b*pb)
                        
                        for i in range(1,K):
                            r,c = access_q(n,s,b, n-i,s+i,b-1)
                            q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,i)*pb)
                        
                        r,c = access_q(n,s,b, n-K,s+K,b-1)
                        q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,K))
                
    
                elif n > 0 and F > 0 and F < (T-1):
                    K = min(F,n-1)
                    if s > 0:
                        r,c = access_q(n,s,b, n,s-1,b)
                        q[r,c] = q[r,c] + (s*mu_s)
                    if b > 0:
                        for i in range(0,K):
                            r,c = access_q(n,s,b, n-i-1,s+i,b)
                            q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,i)*pb)
                            
                        r,c = access_q(n,s,b, n-K-1,s+K,b)
                        q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,K))
                    
                elif n > 0 and F == (T-1):
                    K = min(F,n-1)
                    if s > 0:
                        r,c = access_q(n,s,b, n-1,s-1,b+1)
                        q[r,c] = q[r,c] + (s*mu_s)
                    if b > 0:
                        for i in range(0,K):
                            r,c = access_q(n,s,b, n-i-1,s+i,b)
                            q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,i)*pb)
                            
                        r,c = access_q(n,s,b, n-K-1,s+K,b)
                        q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,K))
                        
                elif n == 0 and (s+(b*T)) <= N:
                    if s > 0:
                        r,c = access_q(0,s,b, 0,s-1,b)
                        q[r,c] = q[r,c] + (s*mu_s)
                    if b > 0:
                        r,c = access_q(0,s,b, 0,s,b-1)
                        q[r,c] = q[r,c] + (b*mu_b)

def fill_diagonal():
    for i in range(q.shape[0]):
        q[i,i] = -(np.sum(q[i]))
        
init_transitions_arr(T+2)
init_transitions_srv(T+2)
fill_diagonal()

diag = d1 + (d2*T)

Bs = []
for i in range(1,T+1):
    block = q[diag:diag+d2, diag-(d2*i):diag-(d2*i)+d2]
    Bs.append(block)
    
L = q[diag:diag+d2, diag:diag+d2]
F = q[diag:diag+d2, diag+d2:diag+d2+d2]
L_prime = q[0:d1,0:d1]
F_prime = q[0:d1,d1:d1+d2]
Bs_prime = [q[d1:d1+d2,0:d1]]
for i in range(T-1):
    Bs_prime.append(q[d1+((i+1)*d2) : d1+((i+1)*d2)+d2 , 0:d1])

L_inv = np.linalg.inv(L)
V = np.matmul(F,L_inv)
Ws = []
for B in Bs:
    W = np.matmul(B,L_inv)
    Ws.append(W)
Ws

Rs = [np.zeros((d2,d2))]
diff = 1
i = 0
while diff > 1e-20:    
    R = -V
    j = 2
    for W in Ws:
        R = np.subtract(R,np.matmul(np.linalg.matrix_power(Rs[i],j),W))
        j += 1
    diff = R[0][0] - Rs[-1][0][0]
    Rs.append(R)
    i += 1
print(len(Rs))
R = Rs[-1]

#
tl = L_prime
tr = F_prime
bl = Bs_prime[0]
for i in range(1,len(Bs_prime)):
    b = Bs_prime[i]
    bl = np.add(bl,np.matmul(np.linalg.matrix_power(R,i),b))
br = L
#i = 1
for i in range(len(Bs)):
    b = Bs[i]
    br = np.add(br,np.matmul(np.linalg.matrix_power(R,i+1),b))
'''for b in Bs:
    br = np.add(br,np.matmul(np.linalg.matrix_power(R,i),b))
    i += 1'''


#
bc = np.append(tl,tr,axis=1)
bcb = np.append(bl,br,axis=1)
bc = np.append(bc,bcb,axis=0)
bc[0][-1] = 1
for r in bc[1:]:
    r[-1] = 0

#
a = np.transpose(bc)
b = np.zeros(d1+d2)
b[-1] = 1
pi = np.linalg.solve(a,b)

#
ir = np.linalg.inv(np.subtract(np.identity(R.shape[0]),R))
r = np.matmul(pi[d1:],ir)
alfa = np.sum(pi[:d1]) + np.sum(r)
pi = pi/alfa
pis = [pi[:d1],pi[d1:]]
for i in range(2,101):
    pis.append(np.matmul(pis[i-1],R))
    
print("Total time --- %s seconds ---" % (time.time() - start_sim))

In [92]:
## NEW

N = 400
T = 200
arr_rate = 1
pb = 0.1
ps = 1-pb
lambda_s = (arr_rate*ps)
lambda_b = (arr_rate*pb)
mu_s = 1
mu_b = 0.1

start_sim = time.time()

d1 = round(N + ((N*N)/(2*T)) - (N/2) + (N/T) + 1)
d2 = round((T)*(N/T) + 1)
#print(d1)
#print(d2)

q_dim = d1+(d2*(T+2))
q = lil_matrix((q_dim,q_dim))
#q = csc_matrix(q)
#print(q)

def access_q(n1,s1,b1,n2,s2,b2):
    global q
    if n1 == 0:
        pass
    elif n2 == 0:
        pass
    elif n1 == T+1:
        pass
    else:
        return ((0,0))
    
    q_dim = ((q.shape[0]-d1)/d2)

    row = access_map[(s1,b1)]
    col = access_map[(s2,b2)]
    if (n1>0):
        row = d1 + ((n1-1)*d2) + access_map_v2[(s1,b1)]
    if (n2>0):
        col = d1 + ((n2-1)*d2) + access_map_v2[(s2,b2)]
    return((row,col))


access_map = dict()
i = 0
for s in range(N+1):
    for b in range(round(N/T)+1):
        if (s+(b*T)) <= N:
            access_map[(s,b)] = i
            i = i+1

access_map_v2 = dict()
i = 0
for s in range(N+1):
    for b in range(round(N/T)+1):
        if (s+(b*T)) > (N-T) and (s+(b*T)) <= N:
            access_map_v2[(s,b)] = i
            i = i+1

def init_transitions_arr(dim):
    global q
    for n in range(dim):
        for s in range(N+1):
            for b in range (N+1):
                if n > 0 and (s,b) in access_map_v2:
                    r,c = access_q(n,s,b, n+1,s,b)
                    q[r,c] = arr_rate
                elif n == 0:
                    if (s+(b*T)) == N:
                        r,c = access_q(0,s,b, 1,s,b)
                        q[r,c] = arr_rate
                    elif (s+(b*T)) <= (N-T):
                        r,c = access_q(0,s,b, 0,s+1,b)
                        q[r,c] = lambda_s
                        r,c = access_q(0,s,b, 0,s,b+1)
                        q[r,c] = lambda_b
                    elif (s+(b*T)) > (N-T) and (s+(b*T)) < N:
                        r,c = access_q(0,s,b, 0,s+1,b)
                        q[r,c] = lambda_s
                        r,c = access_q(0,s,b, 1,s,b)
                        q[r,c] = lambda_b

def init_transitions_srv(dim):
    global q
    for n in range(dim):
        for s in range(N+1):
            for b in range (N+1):
                F = N - (b*T) - s

                if n > 0 and (s+(b*T)) == N:
                    K = min(n,T)
                    if s > 0:
                        r,c = access_q(n,s,b, n-1,s,b)
                        q[r,c] = q[r,c] + (s*mu_s*ps)

                        r,c = access_q(n,s,b, n,s-1,b)
                        q[r,c] = q[r,c] + (s*mu_s*pb)
                    if b > 0:
                        r,c = access_q(n,s,b, n-1,s,b)
                        q[r,c] = q[r,c] + (b*mu_b*pb)

                        for i in range(1,K):
                            r,c = access_q(n,s,b, n-i,s+i,b-1)
                            q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,i)*pb)

                        r,c = access_q(n,s,b, n-K,s+K,b-1)
                        q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,K))


                elif n > 0 and F > 0 and F < (T-1):
                    K = min(F,n-1)
                    if s > 0:
                        r,c = access_q(n,s,b, n,s-1,b)
                        q[r,c] = q[r,c] + (s*mu_s)
                    if b > 0:
                        for i in range(0,K):
                            r,c = access_q(n,s,b, n-i-1,s+i,b)
                            q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,i)*pb)

                        r,c = access_q(n,s,b, n-K-1,s+K,b)
                        q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,K))

                elif n > 0 and F == (T-1):
                    K = min(F,n-1)
                    if s > 0:
                        r,c = access_q(n,s,b, n-1,s-1,b+1)
                        q[r,c] = q[r,c] + (s*mu_s)
                    if b > 0:
                        for i in range(0,K):
                            r,c = access_q(n,s,b, n-i-1,s+i,b)
                            q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,i)*pb)

                        r,c = access_q(n,s,b, n-K-1,s+K,b)
                        q[r,c] = q[r,c] + (b*mu_b*math.pow(ps,K))

                elif n == 0 and (s+(b*T)) <= N:
                    if s > 0:
                        r,c = access_q(0,s,b, 0,s-1,b)
                        q[r,c] = q[r,c] + (s*mu_s)
                    if b > 0:
                        r,c = access_q(0,s,b, 0,s,b-1)
                        q[r,c] = q[r,c] + (b*mu_b)

def fill_diagonal():
    for i in range(q.shape[0]):
        q[i,i] = -(np.sum(q[i])-q[i,i])
    

init_transitions_arr(T+2)
init_transitions_srv(T+2)
fill_diagonal()

#q = csr_matrix(q).toarray()

diag = d1 + (d2*T)

Bs = []
for i in range(1,T+1):
    block = q[diag:diag+d2, diag-(d2*i):diag-(d2*i)+d2].toarray()
    Bs.append(block)

L = q[diag:diag+d2, diag:diag+d2].toarray()
F = q[diag:diag+d2, diag+d2:diag+d2+d2].toarray()
L_prime = q[0:d1,0:d1].toarray()
F_prime = q[0:d1,d1:d1+d2].toarray()
Bs_prime = [q[d1:d1+d2,0:d1].toarray()]
for i in range(T-1):
    Bs_prime.append(q[d1+((i+1)*d2) : d1+((i+1)*d2)+d2 , 0:d1].toarray())
    
print("Generating time --- %s seconds ---" % (time.time() - start_sim))

L_inv = np.linalg.inv(L)
V = np.matmul(F,L_inv)
Ws = []
for B in Bs:
    W = np.matmul(B,L_inv)
    Ws.append(W)
Ws

Rs = [np.zeros((d2,d2))]
diff = 1
i = 0
while diff > 1e-12:    
    R = -V
    j = 2
    for W in Ws:
        R = np.subtract(R,np.matmul(np.linalg.matrix_power(Rs[i],j),W))
        j += 1
    diff = R[0][0] - Rs[-1][0][0]
    Rs.append(R)
    i += 1
R = Rs[-1]
print(len(Rs))

#
tl = L_prime
tr = F_prime
bl = Bs_prime[0]
for i in range(1,len(Bs_prime)):
    b = Bs_prime[i]
    bl = np.add(bl,np.matmul(np.linalg.matrix_power(R,i),b))
br = L
#i = 1
for i in range(len(Bs)):
    b = Bs[i]
    br = np.add(br,np.matmul(np.linalg.matrix_power(R,i+1),b))
'''for b in Bs:
    br = np.add(br,np.matmul(np.linalg.matrix_power(R,i),b))
    i += 1'''


#
bc = np.append(tl,tr,axis=1)
bcb = np.append(bl,br,axis=1)
bc = np.append(bc,bcb,axis=0)
bc[0][-1] = 1
for r in bc[1:]:
    r[-1] = 0

#
a = np.transpose(bc)
b = np.zeros(d1+d2)
b[-1] = 1
pi = np.linalg.solve(a,b)

#
ir = np.linalg.inv(np.subtract(np.identity(R.shape[0]),R))
r = np.matmul(pi[d1:],ir)
alfa = np.sum(pi[:d1]) + np.sum(r)
pi = pi/alfa
pis = [pi[:d1],pi[d1:]]
for i in range(2,max(N,100)):
    pis.append(np.matmul(pis[i-1],R))
    
print("Total time --- %s seconds ---" % (time.time() - start_sim))

pis

Generating time --- 33.30751347541809 seconds ---
59
Total time --- 161.64444518089294 seconds ---


[array([1.21587646e-01, 1.08240323e-01, 1.58147691e-02, 1.10763613e-01,
        1.03742637e-01, 5.08620410e-02, 5.26702035e-02, 1.58772835e-02,
        2.03487825e-02, 3.92460474e-03, 7.80810218e-03, 9.10531664e-04,
        3.68849803e-03, 2.60365985e-04, 2.19182195e-03, 1.11985887e-04,
        1.47858445e-03, 6.42124075e-05, 1.05378280e-03, 4.13051211e-05,
        7.71594719e-04, 2.78100572e-05, 5.74619502e-04, 1.92067336e-05,
        4.33255905e-04, 1.35105135e-05, 3.29852568e-04, 9.64348909e-06,
        2.53113662e-04, 6.96700135e-06, 1.95503339e-04, 5.08503641e-06,
        1.51843854e-04, 3.74411850e-06, 1.18495723e-04, 2.77787703e-06,
        9.28534363e-05, 2.07481868e-06, 7.30234088e-05, 1.55890130e-06,
        5.76120235e-05, 1.17747434e-06, 4.55826369e-05, 8.93604109e-07,
        3.61570740e-05, 6.81084368e-07, 2.87465614e-05, 5.21132201e-07,
        2.29025585e-05, 4.00164210e-07, 1.82812910e-05, 3.08278287e-07,
        1.46178729e-05, 2.38203861e-07, 1.17072353e-05, 1.845677

In [94]:
## SOLVING

start_sim = time.time()

mLz = np.array([
    [s[0] for s in access_map],
    [s[1] for s in access_map]
])
mL_t = []
mL_b = []
for s in access_map_v2:
    if s[0] + (s[1]*T) < N:
        mL_t.append(s[0])
        mL_b.append(s[1]+1) #HOL
    elif s[0] + (s[1]*T) == N:
        mL_t.append(s[0] + ps)
        mL_b.append(s[1] + pb)
mL = np.array([
    mL_t,
    mL_b
])

Nl = np.matmul(mLz,pis[0])

Nrl = np.matmul(np.transpose(pis[1]),np.matmul(R,np.linalg.matrix_power(np.subtract(np.identity(R.shape[0]),R),-2)))
Nrl = np.sum(Nrl)
Nrl = np.array([ps,pb])*Nrl

Nrr = np.matmul(np.transpose(pis[1]),np.linalg.inv(np.subtract(np.identity(R.shape[0]),R)))

Nrr = np.matmul(mL,np.transpose(Nrr))

Nr = np.add(Nrl,Nrr)
N_ao = np.add(Nl,Nr)

Nwb = 0
Nws = 0
Nsb = 0
Nss = 0
n = 1
for sp in pis[1:]:
    i = 0
    for s in access_map_v2:
        if s[0] + (s[1]*T) < N:
            Nwb += sp[i]*(1+((n-1)*pb))
            Nws += sp[i]*((n-1)*ps)
        elif s[0] + (s[1]*T) == N:
            Nwb += sp[i]*n*pb
            Nws += sp[i]*n*ps

        Nsb += sp[i]*s[1]
        Nss += sp[i]*s[0]

        i += 1
    n += 1

Nw = np.array([Nws, Nwb])

#Ns for n = 0
i = 0
for s in access_map:
    Nsb += pis[0][i]*s[1]
    Nss += pis[0][i]*s[0]
    i += 1

#N, total
Nb = Nwb+Nsb
Ns = Nws+Nss

#T
RT = np.sum(N_ao)/arr_rate
RTs = N_ao[0]/lambda_s
RTb = N_ao[1]/lambda_b

#print("Run time --- %s seconds ---" % (time.time() - start_sim))


print("Total time --- %s seconds ---" % (time.time() - start_sim))

Total time --- 0.2986307144165039 seconds ---


In [93]:
##SIMULATION

def avl_server(t,servers):
    low = len(servers)
    i = 0
    for s in servers:
        if t > s[1]:
            if low == len(servers):
                low = i
            elif s[1] < servers[low][1]:
                low = i
        i += 1
    if low < len(servers):
        return low #return server with earliest idle time
    return len(servers)

def get_avl_servers(t,servers,n):
    empty = []
    i = 0
    for s in servers:
        if t >= s[1]: #get empty servers
            empty.append((i,s[1]))
        i += 1
    empty.sort(key = lambda x: x[1])
    return empty[:n] #return the [server index,free time]

def busy_server_n(t,servers):
    n = 0
    i = 0
    jobs = dict()
    jobs['big'] = []
    jobs['small'] = []
    jobs[''] = []
    for s in servers:
        if s[1] == 0:
            continue
        if t < s[1] and t >= s[0]: #get busy server
            n += 1
            job_type = s[2].split('_')[0]
            jobs[job_type].append(s[2])
        i+=1
    big_job_n = len(list(set(jobs['big'])))
    small_job_n = len(list(set(jobs['small'])))
    return (n,small_job_n,big_job_n) #reverse

def queue_counter(q,big_job_size):
    small_jobs = 0
    big_jobs = 0
    for job in q:
        if job[1] == big_job_size:
            big_jobs += 1
        else:
            small_jobs += 1
    return ((small_jobs,big_jobs))

def get_job_size(max_size, pb):
    limit = pb*100
    randn = random.randint(1,100)
    if randn > limit:
        return 1
    return max_size

def get_srv_rate(job_size):
    if job_size == 1:
        return mu_s
    elif job_size == T:
        return mu_b

measurements = []
for run in range(20):
    start_sim = time.time()
    max_job_size = T
    server_start = 0

    duration = 3600*100
    #job_size = 10

    server_n = N
    #srv_rate = 5

    # POPULATE THE JOBS ARRIVAL TIMES AND JOB SIZE
    t = 0
    jobs_arr_t = []
    while t <= duration:
        iat = np.random.exponential(1/arr_rate)
        t += iat
        #jobs_arr_t.append((t,random.choice([1,job_size])))
        jobs_arr_t.append((t,get_job_size(max_job_size,pb)))

    waiting_times = []
    response_times = []
    queue = []

    # INITIATE THE N SERVERS
    servers = [] #[service_start,end]
    for s in range(server_n):
        servers.append([0,server_start,''])

    server_load = [[0,0]]
    queue_size = [[0,0]]
    jobs_inside = [[0,0,0]]
    jobs_queue = [[0,0,0]]
    throughput_small = 0
    throughput_big = 0
    nsb = [(0,0,0,0)]

    job_id = 0
    # ITERATE EVENT-BASED (CAN BE JOBS ARRIVAL OR SERVICE FINISHED)
    while job_id < len(jobs_arr_t):
        t = 999999999
        curr_job = jobs_arr_t[job_id] 
        # ITERATE ALL THE SERVERS AND GET THE NEXT EARLIEST SERVER FINISH
        for s in servers:
            if s[1] > 0 and s[1] < t and s[1] > server_load[-1][0]:
                t = s[1]
        # GET THE NEXT JOBS ARRIVAL
        if jobs_arr_t[job_id][0] < t:
            t = jobs_arr_t[job_id][0]
            queue.append(curr_job)
            job_id += 1
        # NOW, t IS THE NEXT EVENT HAPPENING
        #print(t)

        # HOL TECHNIQUE WHERE SCHEDULERS PRIORITIZE THE JOBS AT THE FRONT,
        # AND WAIT UNTIL HOWEVER MANY SERVERS THAT JOB NEEDS ARE AVAILABLE
        while len(queue) > 0:
            job = queue[0] #only the job at the front
            avl_servers_n = len(servers)-busy_server_n(t,servers)[0] #get the number of available servers
            if avl_servers_n >= job[1]: #if it's enough for the job at the front
                ready_servers = get_avl_servers(t,servers,job[1]) #get all the servers ready to service
                st = np.random.exponential(1/get_srv_rate(job[1])) #determine the service rate
                for s in ready_servers:# put the job in each ready server
                    server_id = s[0]
                    job_name = ''
                    if job[1] == max_job_size: #for graph purpose
                        job_name += 'big'
                    else:
                        job_name += 'small'
                    job_name += '_'+str(job[0])
                    servers[server_id] = [t,t+st,job_name]
                waiting_times.append((t-job[0],job[1])) #for graph purpose
                response_times.append((t+st-job[0],job[1])) #for graph purpose
                queue.pop(0)
                if job[1] == max_job_size: #for graph purpose
                    throughput_big += 1
                else:
                    throughput_small += 1
            else:
                break

        server_state = busy_server_n(t,servers)
        queue_state = queue_counter(queue,max_job_size)
        server_load.append((t,server_state[0])) #for graph purpose
        queue_size.append((t,len(queue))) #for graph purpose
        jobs_inside.append((t,server_state[1],server_state[2]))
        jobs_queue.append((t,queue_state[0],queue_state[1]))
        nsb.append((t,len(queue),server_state[1],server_state[2]))

    print('Arrival rate: '+str(arr_rate))
    
    avg_wait = []
    n = 1
    tot = 0
    for w in [wt[0] for wt in waiting_times]:
        tot += w
        avg_wait.append(tot/n)
        n += 1

    avg_resp = []
    avg_resp_s = 0
    avg_resp_b = 0
    n = 1
    tot = 0
    for w in [rt[0] for rt in response_times]:
        tot += w
        avg_resp.append(tot/n)
        n += 1
        
    for rt in response_times:
        if rt[1] == 1:
            avg_resp_s += rt[0]
        else:
            avg_resp_b += rt[0]
    avg_resp_s = avg_resp_s/throughput_small
    avg_resp_b = avg_resp_b/throughput_big

    plt.figure(figsize=(15,10))
    plt.plot([i for i in range(len(avg_wait))], avg_wait, label = "Wait Times")
    plt.plot([i for i in range(len(avg_resp))], avg_resp, label = "Response Times")
    plt.legend()
    plt.show()

    # counting the utilization
    util = 0 
    n_small = 0
    n_big = 0
    n_small_w = 0
    n_big_w = 0
    i = 1
    while i < len(server_load):
        if server_load[i-1][0] < server_start:
            i += 1
            continue
        timestamp = server_load[i][0] - server_load[i-1][0]
        load = server_load[i-1][1]
        small_load = jobs_inside[i-1][1]
        big_load = jobs_inside[i-1][2]
        small_queue = jobs_queue[i-1][1]
        big_queue = jobs_queue[i-1][2]
        util += (timestamp*load)
        n_small += (timestamp*small_load)
        n_big += (timestamp*big_load)
        n_small_w += (timestamp*small_queue)
        n_big_w += (timestamp*big_queue)
        i += 1
    duration -= server_start
    avg_util = util/duration/server_n
    avg_n_small = n_small/duration
    avg_n_big = n_big/duration
    avg_n_small_w = n_small_w/duration
    avg_n_big_w = n_big_w/duration
    avg_thr_small = throughput_small/duration
    avg_thr_big = throughput_big/duration
    measurements.append([nsb,avg_n_small,avg_n_big,avg_n_small_w,avg_n_big_w,avg_resp[-1],avg_resp_s,avg_resp_b])

    print('Utilization: '+str(avg_util))
    print('Small job throughput: '+str(avg_thr_small))
    print('Big job throughput: '+str(avg_thr_big))
    print('Average number of small job in srv: '+str(avg_n_small))
    print('Average number of big job in srv: '+str(avg_n_big))
    print('Average number of small job in waiting: '+str(avg_n_small_w))
    print('Average number of big job in waiting: '+str(avg_n_big_w))
    print('Average number of small job total: '+str(avg_n_small+avg_n_small_w))
    print('Average number of big job total: '+str(avg_n_big+avg_n_big_w))

    print("Total time --- %s seconds ---" % (time.time() - start_sim))
    print('------------')
    
nsbs = []
for m in measurements:
    nsb = m[0]
    stat = dict()
    last_t = 0
    last_st = (0,0,0)
    for st in nsb[1:]:
        if last_st in stat:
            stat[last_st] += (st[0]-last_t)
        else:
            stat[last_st] = (st[0]-last_t)
        last_t = st[0]
        last_st = (st[1],st[2],st[3])

    for st in stat:
        stat[st] = stat[st]/duration
        
    nsbs.append(stat)

KeyboardInterrupt: 

In [90]:
## VALIDATION

conf_num = 0.95

measures = ['N - small', 'N - big', 'Nw- small', 'Nw - big', 'Rt', 'Rt- small', 'Rt - big']

table = [['Measure', 'Matrix geom.', 'Sim. Estimate', 'Confidence Interval', 'Check']]
for i in range(1,3):
    sim_data = [m[i]+m[i+2] for m in measurements]
    mean = np.mean(sim_data)
    conf = sct.t.interval(alpha=conf_num, df=len(sim_data)-1, loc=mean, scale=sct.sem(sim_data))
    check = False
    if N_ao[i-1] > conf[0] and N_ao[i-1] < conf[1]:
        check = True
    table.append([measures[i-1], N_ao[i-1], mean, conf, check])
    
for i in range(3,5):
    sim_data = [m[i] for m in measurements]
    mean = np.mean(sim_data)
    conf = sct.t.interval(alpha=conf_num, df=len(sim_data)-1, loc=mean, scale=sct.sem(sim_data))
    check = False
    if Nw[i-3] > conf[0] and Nw[i-3] < conf[1]:
        check = True
    table.append([measures[i-1], Nw[i-3], mean, conf, check])
    
for i in range(5,6):
    sim_data = [m[i] for m in measurements]
    mean = np.mean(sim_data)
    conf = sct.t.interval(alpha=conf_num, df=len(sim_data)-1, loc=mean, scale=sct.sem(sim_data))
    check = False
    resp_t = np.sum(N_ao)/arr_rate
    if resp_t > conf[0] and resp_t < conf[1]:
        check = True
    table.append([measures[i-1], resp_t, mean, conf, check])
    
for i in range(6,8):
    sim_data = [m[i] for m in measurements]
    mean = np.mean(sim_data)
    conf = sct.t.interval(alpha=conf_num, df=len(sim_data)-1, loc=mean, scale=sct.sem(sim_data))
    check = False
    lam = lambda_s
    if i == 7:
        lam = lambda_b
    resp_t = N_ao[i-6]/lam
    if resp_t > conf[0] and resp_t < conf[1]:
        check = True
    table.append([measures[i-1], resp_t, mean, conf, check])

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒═══════════╤════════════════╤═════════════════╤══════════════════════════════════════════╤═════════╕
│ Measure   │   Matrix geom. │   Sim. Estimate │ Confidence Interval                      │ Check   │
╞═══════════╪════════════════╪═════════════════╪══════════════════════════════════════════╪═════════╡
│ N - small │       5.20795  │        5.22085  │ (5.160069423325842, 5.281625624089257)   │ True    │
├───────────┼────────────────┼─────────────────┼──────────────────────────────────────────┼─────────┤
│ N - big   │       1.56516  │        1.56298  │ (1.5533618649132879, 1.572598450205129)  │ True    │
├───────────┼────────────────┼─────────────────┼──────────────────────────────────────────┼─────────┤
│ Nw- small │       4.30795  │        4.32122  │ (4.260282254949553, 4.382160785484219)   │ True    │
├───────────┼────────────────┼─────────────────┼──────────────────────────────────────────┼─────────┤
│ Nw - big  │       0.565158 │        0.564416 │ (0.5563577881821714, 0.5724743562

In [95]:
res = [['State','Matrix Geom.','Simulation','Diff', 'Check']]
#res = [['State','Truncated matrix']]
i = 0
for st in access_map:
    mg = pis[0][i]
    
    sim_data = [0,0,0,0,0]
    try:
        sim_data = [nsb[(0,st[0],st[1])] for nsb in nsbs]
    except KeyError:
        i += 1
        continue
    
    mean = np.mean(sim_data)
    conf = sct.t.interval(alpha=0.95, df=len(sim_data)-1, loc=mean, scale=sct.sem(sim_data))
    check = True
    diff = '-'
    if mg < conf[0]:
        check = False
        diff = '{:.1%}'.format((conf[0]-mg)/mg)
    elif mg > conf[1]:
        check = False
        diff = '{:.1%}'.format((mg-conf[1])/mg)
    
    res.append([(0,st[0],st[1]),mg,conf,diff, check])
    #res.append([[0,st[0],st[1]],trunc_pis[i]])
    i += 1

for j in range(1,len(pis)):
    i = 0
    for st in access_map_v2:
        mg = pis[j][i]
        
        sim_data = [0,0,0,0,0]
        try:
            sim_data = [nsb[(j,st[0],st[1])] for nsb in nsbs]
        except KeyError:
            i += 1
            continue
        mean = np.mean(sim_data)
        conf = sct.t.interval(alpha=0.95, df=len(sim_data)-1, loc=mean, scale=sct.sem(sim_data))
        check = True
        diff = '-'
        if mg < conf[0]:
            check = False
            diff = '{:.1%}'.format((conf[0]-mg)/mg)
        elif mg > conf[1]:
            check = False
            diff = '{:.1%}'.format((mg-conf[1])/mg)

        res.append([(j,st[0],st[1]),mg, conf,diff, check])
        #res.append([[j,st[0],st[1]],trunc_pis[d1+((j-1)*5)+i]])
        i += 1
    
print(tabulate(res, headers='firstrow', tablefmt='fancy_grid'))

╒═════════════╤════════════════╤══════════════════════════════════════════════════╤════════╤═════════╕
│ State       │   Matrix Geom. │ Simulation                                       │ Diff   │ Check   │
╞═════════════╪════════════════╪══════════════════════════════════════════════════╪════════╪═════════╡
│ (0, 0, 0)   │    0.121588    │ (0.1212806842647872, 0.12259938248190662)        │ -      │ True    │
├─────────────┼────────────────┼──────────────────────────────────────────────────┼────────┼─────────┤
│ (0, 0, 1)   │    0.10824     │ (0.1080733263673099, 0.108913694948202)          │ -      │ True    │
├─────────────┼────────────────┼──────────────────────────────────────────────────┼────────┼─────────┤
│ (0, 0, 2)   │    0.0158148   │ (0.015674921919983854, 0.0160151516327175)       │ -      │ True    │
├─────────────┼────────────────┼──────────────────────────────────────────────────┼────────┼─────────┤
│ (0, 1, 0)   │    0.110764    │ (0.11060812930994951, 0.1116802853405162